In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import joblib


face_model = joblib.load('face_model.pkl')
face_scaler = joblib.load('face_model_scaler.pkl')
hand_model = joblib.load('hand_model.pkl')
hand_scaler = joblib.load('hand_model_scaler.pkl')

mp_face = mp.solutions.face_mesh.FaceMesh()
mp_hand = mp.solutions.hands.Hands(max_num_hands=1)
mp_draw = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break

    frame = cv2.flip(frame, 1)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    face_result = mp_face.process(rgb)
    hand_result = mp_hand.process(rgb)

    face_label = None
    hand_label = None

    if face_result.multi_face_landmarks:
        face_landmarks = face_result.multi_face_landmarks[0].landmark
        face_input = np.array([[lm.x, lm.y] for lm in face_landmarks]).flatten().reshape(1, -1)
        face_input_scaled = face_scaler.transform(face_input)
        face_label = face_model.predict(face_input_scaled)[0]
        mp_draw.draw_landmarks(frame, face_result.multi_face_landmarks[0], mp.solutions.face_mesh.FACEMESH_CONTOURS)


    if hand_result.multi_hand_landmarks:
        hand_landmarks = hand_result.multi_hand_landmarks[0].landmark
        hand_input = np.array([[lm.x, lm.y] for lm in hand_landmarks]).flatten().reshape(1, -1)
        hand_input_scaled = hand_scaler.transform(hand_input)
        hand_label = hand_model.predict(hand_input_scaled)[0]
        mp_draw.draw_landmarks(frame, hand_result.multi_hand_landmarks[0], mp.solutions.hands.HAND_CONNECTIONS)

    
    sentence = ""
    if face_label and hand_label:
        sentence = f"You show {face_label} expression and {hand_label} gesture"
    elif face_label:
        sentence = f"You show {face_label} expression"
    elif hand_label:
        sentence = f"You show {hand_label} gesture"

    if sentence:
        cv2.putText(frame, sentence, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)

    cv2.imshow("Real-Time Face & Hand Gesture Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
